In [5]:
import vtk
from typing import Literal
import numpy as np
from vtk.util import numpy_support

def read_polydata(file_path):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    return reader.GetOutput()

def write_polydata(polydata, file_path):
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(file_path)
    writer.SetInputData(polydata)
    writer.SetDataModeToAscii()
    writer.Write()

def create_envelope(polydata_list: list[vtk.vtkPolyData], mode: Literal["min", "max"]):
    if not polydata_list:
        raise ValueError("The list of polydata objects is empty")

    # Assume all polydata have the same structure
    reference_polydata = polydata_list[0]
    num_cells = reference_polydata.GetNumberOfCells()

    # Create a new polydata to store the envelope
    envelope_polydata = vtk.vtkPolyData()
    envelope_polydata.DeepCopy(reference_polydata)
    array_name = reference_polydata.GetCellData().GetArray(0).GetName()   
    
    # cell_data = reference_polydata.GetCellData().GetScalars()
    # if cell_data is None:
    #     raise ValueError("The reference polydata does not contain scalar data")

    # Iterate through each cell and find the maximum value
    if mode == "max":
        env_values = np.full(num_cells, -np.inf)
        env_function = np.maximum
    elif mode == "min":
        env_values = np.full(num_cells, np.inf)
        env_function = np.minimum
    else:
        raise Exception("Mode not available")
        
    # Iterate through each polydata and update the max_values array
    for polydata in polydata_list:
        cell_data = polydata.GetCellData().GetArray(0)
        values = numpy_support.vtk_to_numpy(cell_data)
        env_values = env_function(env_values, values)
        
    # Convert the max_values array back to a vtkArray
    env_values_vtk = numpy_support.numpy_to_vtk(env_values)
    env_values_vtk.SetName(array_name)

    # Set the maximum values to the envelope polydata
    envelope_polydata.GetCellData().SetScalars(env_values_vtk)

    return envelope_polydata

In [6]:
import pathlib

# project_path = pathlib.Path("/mnt/disk01/insight-volume/V0/Eztec")
project_path = pathlib.Path("/mnt/disk01/insight-volume/V0/PrologisCajamar4")

# subproject_name = "building"
subproject_name = "G100"
# subproject_name = "G200"
# subproject_name = "G300"

# directions = ["000", "023", "045", "068", "090", "113", "135", "158", "180", "203", "225", "270", "315", "338"]
# statistics = ["mean", "rms", "min", "max"]
# datasets = ["Cf_floors_x", "Cf_floors_y", "Cm_floors_z", "cp_root"]

directions = ["013", "058", "103", "148", "193", "238", "283", "328"]
# directions = ["000", "045", "090", "135", "180", "225", "270", "315"]
statistics = ["mean", "rms", "min", "max", "mean_eq"]
# datasets = ["cp_root"]
datasets = ["Ce_roof"]

for dataset in datasets:
    for stat in statistics:
        file_paths = [project_path / subproject_name / "cases" / direction / dataset / f"{stat}.vtp" for direction in directions]
        
        polydata_list = [read_polydata(f_path) for f_path in file_paths]
        for mode in ["min", "max"]:
            env_polydata = create_envelope(polydata_list, mode)
            (project_path / subproject_name / "cases" / f"envelope_{mode}" / dataset).mkdir(parents=True, exist_ok=True)
            write_polydata(env_polydata, project_path / subproject_name / "cases" / f"envelope_{mode}" / dataset / f"{stat}.vtp")